In [1]:
# !pip install --upgrade --quiet langchain-community unstructured openpyxl langchain_groq pydantic langchain_mistralai
# !pip install -qU trustcall
# !pip install -qU langchain_openai

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from ultralytics.utils.benchmarks import benchmark


In [3]:
import os
import re
import math
import json
import pandas as pd
import numpy as np
import ast  # Import the ast module
from typing import List, Dict, Any, Literal
from collections import Counter
from pydantic import BaseModel, Field

from langchain.output_parsers import PydanticOutputParser
# from langchain_groq import ChatGroq
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
# from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # Import ChatPromptTemplate
#from google.colab import userdata

In [4]:
# df = pd.read_excel(userdata.get('MY_PATH_EXCEL'))
import pandas as pd
import json
import calendar

# تحميل البيانات من ملف json
with open("/home/elbahnasy/CodingWorkspace/Hands-On-LLMS/notebooks/employees_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

employees = data["employees"]

# جلب السنة والشهر من ملف json نفسه
year = data["year"]
month = data["month"]

# جلب عدد أيام الشهر تلقائياً
_, num_days = calendar.monthrange(year, month)
dates = [f"{year}-{str(month).zfill(2)}-{str(day).zfill(2)}" for day in range(1, num_days + 1)]

rows = []
for emp in employees:
    row = {
        "user_id": emp["user_id"],
        "name": emp["name"],
        "role": emp["role"]
    }
    # dict للتفضيلات زي ما هي في الـjson
    pref_dict = {p["date"]: p["shiftType"] for p in emp["preferences"]}
    for date in dates:
        row[date] = pref_dict.get(date, "")
    rows.append(row)

# إنشاء DataFrame
data = pd.DataFrame(rows)

# معاينة أول 5 صفوف
data.head(10)

,user_id,name,role,2025-08-01,2025-08-02,2025-08-03,2025-08-04,2025-08-05,2025-08-06,2025-08-07,...,2025-08-22,2025-08-23,2025-08-24,2025-08-25,2025-08-26,2025-08-27,2025-08-28,2025-08-29,2025-08-30,2025-08-31
0,2,Emp E1D1#1,Manager,,,,,,,,...,,,,,,,,,,
1,3,Emp E1D1#2,Manager,,,,,,,,...,,,,,,,,,,
2,4,Emp E1D1#3,Manager,,,,,,,,...,,,,,,,,,,
3,5,Emp E1D1#4,Manager,,,,,,,,...,,,,,,,,,,
4,6,Emp E1D1#5,Employee,night,,,,,,morning,...,,,,,,,,,,
5,7,Emp E1D1#6,Employee,afternoon,,,,,,off,...,,,morning,afternoon,afternoon,,,night,,night
6,8,Emp E1D1#7,Employee,,afternoon,,morning,,,,...,morning,,,,night,morning,,,,
7,9,Emp E1D1#8,Employee,,,,,,morning,off,...,,,,,,,,,,night
8,10,Emp E1D1#9,Employee,afternoon,afternoon,,morning,,afternoon,off,...,afternoon,,afternoon,night,morning,,off,,off,afternoon
9,11,Emp E1D1#10,Employee,,off,,,,,,...,,,,,night,afternoon,night,,night,


In [5]:
dic_pref = {}
for date in data.columns[3:]:
    # Use the element-wise OR operator (|) and check for empty strings and "off" explicitly
    dic_pref[date] = data[~((data[date].isna()) | (data[date] == "") | (data[date] == "off"))]['name'].tolist()

dic_pref

{'2025-08-01': ['Emp E1D1#5',
  'Emp E1D1#6',
  'Emp E1D1#9',
  'Emp E1D1#13',
  'Emp E1D1#25',
  'Emp E1D1#37',
  'Emp E1D1#38'],
 '2025-08-02': ['Emp E1D1#7',
  'Emp E1D1#9',
  'Emp E1D1#11',
  'Emp E1D1#12',
  'Emp E1D1#21',
  'Emp E1D1#22',
  'Emp E1D1#24',
  'Emp E1D1#30',
  'Emp E1D1#32',
  'Emp E1D1#33',
  'Emp E1D1#39',
  'Emp E1D1#44'],
 '2025-08-03': ['Emp E1D1#13',
  'Emp E1D1#18',
  'Emp E1D1#21',
  'Emp E1D1#22',
  'Emp E1D1#23',
  'Emp E1D1#31',
  'Emp E1D1#33',
  'Emp E1D1#34',
  'Emp E1D1#37',
  'Emp E1D1#43',
  'Emp E1D1#47',
  'Emp E1D1#49',
  'Emp E1D1#50'],
 '2025-08-04': ['Emp E1D1#7',
  'Emp E1D1#9',
  'Emp E1D1#11',
  'Emp E1D1#14',
  'Emp E1D1#18',
  'Emp E1D1#19',
  'Emp E1D1#23',
  'Emp E1D1#25',
  'Emp E1D1#31',
  'Emp E1D1#33',
  'Emp E1D1#35',
  'Emp E1D1#39',
  'Emp E1D1#40',
  'Emp E1D1#46',
  'Emp E1D1#49'],
 '2025-08-05': ['Emp E1D1#14',
  'Emp E1D1#20',
  'Emp E1D1#24',
  'Emp E1D1#31',
  'Emp E1D1#33',
  'Emp E1D1#35',
  'Emp E1D1#42',
  'Emp E1D1#45'

In [ ]:
import random

def fill_missing_shifts(df):
    shift_choices = ["morning", "afternoon", "night"]

    # الأعمدة اللي هي تواريخ الأيام (كلها بشكل YYYY-MM-DD)
    date_columns = [tatus": nullcol for col in df.columns if col.count('-') == 2]

    # مر على كل خلية في الأعمدة دي
    for col in date_columns:
        df[col] = df[col].apply(
            lambda x: random.choice(shift_choices) if (pd.isna(x) or x == "" or x == "off") else x
        )
    return df

# الاستخدام:
df_filled = fill_missing_shifts(data.copy())
df_filled.head(10)

,user_id,name,role,2025-08-01,2025-08-02,2025-08-03,2025-08-04,2025-08-05,2025-08-06,2025-08-07,...,2025-08-22,2025-08-23,2025-08-24,2025-08-25,2025-08-26,2025-08-27,2025-08-28,2025-08-29,2025-08-30,2025-08-31
0,2,Emp E1D1#1,Manager,night,night,night,afternoon,afternoon,afternoon,morning,...,afternoon,night,morning,morning,night,morning,afternoon,afternoon,morning,morning
1,3,Emp E1D1#2,Manager,afternoon,morning,morning,night,night,morning,morning,...,afternoon,morning,night,afternoon,night,night,night,night,afternoon,night
2,4,Emp E1D1#3,Manager,afternoon,morning,morning,afternoon,night,afternoon,afternoon,...,night,night,morning,afternoon,afternoon,morning,morning,night,afternoon,afternoon
3,5,Emp E1D1#4,Manager,afternoon,morning,afternoon,afternoon,morning,morning,morning,...,afternoon,morning,night,morning,morning,afternoon,night,night,morning,morning
4,6,Emp E1D1#5,Employee,night,night,afternoon,morning,morning,morning,morning,...,morning,night,morning,afternoon,night,afternoon,morning,morning,afternoon,night
5,7,Emp E1D1#6,Employee,afternoon,morning,afternoon,morning,afternoon,night,night,...,morning,afternoon,morning,afternoon,afternoon,afternoon,night,night,night,night
6,8,Emp E1D1#7,Employee,morning,afternoon,night,morning,morning,afternoon,afternoon,...,morning,night,morning,night,night,morning,morning,afternoon,night,night
7,9,Emp E1D1#8,Employee,night,morning,afternoon,afternoon,afternoon,morning,night,...,morning,night,afternoon,night,morning,afternoon,night,morning,afternoon,night
8,10,Emp E1D1#9,Employee,afternoon,afternoon,morning,morning,afternoon,afternoon,afternoon,...,afternoon,afternoon,afternoon,night,morning,morning,morning,morning,morning,afternoon
9,11,Emp E1D1#10,Employee,afternoon,afternoon,night,night,morning,afternoon,morning,...,morning,afternoon,afternoon,afternoon,night,afternoon,night,afternoon,night,afternoon


In [7]:
data.columns

Index(['user_id', 'name', 'role', '2025-08-01', '2025-08-02', '2025-08-03',
       '2025-08-04', '2025-08-05', '2025-08-06', '2025-08-07', '2025-08-08',
       '2025-08-09', '2025-08-10', '2025-08-11', '2025-08-12', '2025-08-13',
       '2025-08-14', '2025-08-15', '2025-08-16', '2025-08-17', '2025-08-18',
       '2025-08-19', '2025-08-20', '2025-08-21', '2025-08-22', '2025-08-23',
       '2025-08-24', '2025-08-25', '2025-08-26', '2025-08-27', '2025-08-28',
       '2025-08-29', '2025-08-30', '2025-08-31'],
      dtype='object')

In [9]:
# Save rows where Title is "Head" to a new variable
heads_df = df_filled[df_filled["role"] == "Manager"]

# Drop those rows from the original DataFrame
df_without_head = df_filled[df_filled["role"] != "Manager"]

In [10]:
df_without_head["role"].value_counts()

role
Employee    46
Name: count, dtype: int64

In [11]:
df_without_head.head()

,user_id,name,role,2025-08-01,2025-08-02,2025-08-03,2025-08-04,2025-08-05,2025-08-06,2025-08-07,...,2025-08-22,2025-08-23,2025-08-24,2025-08-25,2025-08-26,2025-08-27,2025-08-28,2025-08-29,2025-08-30,2025-08-31
4,6,Emp E1D1#5,Employee,night,night,afternoon,morning,morning,morning,morning,...,morning,night,morning,afternoon,night,afternoon,morning,morning,afternoon,night
5,7,Emp E1D1#6,Employee,afternoon,morning,afternoon,morning,afternoon,night,night,...,morning,afternoon,morning,afternoon,afternoon,afternoon,night,night,night,night
6,8,Emp E1D1#7,Employee,morning,afternoon,night,morning,morning,afternoon,afternoon,...,morning,night,morning,night,night,morning,morning,afternoon,night,night
7,9,Emp E1D1#8,Employee,night,morning,afternoon,afternoon,afternoon,morning,night,...,morning,night,afternoon,night,morning,afternoon,night,morning,afternoon,night
8,10,Emp E1D1#9,Employee,afternoon,afternoon,morning,morning,afternoon,afternoon,afternoon,...,afternoon,afternoon,afternoon,night,morning,morning,morning,morning,morning,afternoon


### Prompt

In [ ]:
llm_prompt = """
You are a scheduling assistant.\
Your task is to fairly select exactly **{target_number}** employees from the list below **based on their shift preference for a specific day**.
---
## Variables:
- target_number: the number of employees to select.
- selected_day: "{selected_day}"  # Format: YYYY-MM-DD (e.g. "2025-08-01")
- required_shift: "{required_shift}"
---
## employee_list:This is the full list of employees you are allowed to choose from:{employee_list}
---
## preference_data:This shows the **shift preferences of all employees**, where each key is a date in YYYY-MM-DD format (e.g. "2025-08-01").\
Each object contains the preferred shift per day (column) for that employee.**All day keys are dates, not day names.**
{preferance}
---
## Selection Criteria:
1. Only select employees whose preference for `{selected_day}` (formatted as YYYY-MM-DD) equals `{required_shift}`.
2. Ensure fairness — try to distribute selections among all qualified employees over time (assume past data is tracked externally).
3. Only pick employees from the `employee_list`.
4. If fewer than `target_number` employees match the criteria, return as many as possible.
5. If more than `target_number` match, select any fair subset (random or balanced).
6. Always return exactly {target_number} employees.
   - If fewer than {target_number} employees match the preference criteria, fill the remaining slots with any others from the employee list.
   - If more than {target_number} employees match the preference criteria, only select the {target_number} employees only.
7. Select the {target_number} employees with the highest preference score for this shift, if such a score is present.
---
## Output Format:Return the selected names as a JSON array (no extra text).
## Format Instructions:
{format_instructions}
```json
"""

### Get preferance by names

In [13]:
def get_week_dates_from_day(df, start_day, window_size=7):
    """
    ترجع list بأيام الأسبوع (من start_day) + اليوم اللي قبله لو مش أول يوم في الجدول.
    """
    date_columns = [col for col in df.columns if col.count('-') == 2]
    idx = date_columns.index(start_day)
    week_dates = date_columns[idx:idx+window_size]
    if idx > 0:
        week_dates = [date_columns[idx - 1]] + week_dates
    return week_dates

In [14]:
def get_preferance_employees(df, employee_names: list, date_range: list):
    """
    - df: الداتا فريم الأصلي
    - employee_names: لستة الموظفين
    - date_range: لستة الأعمدة (تواريخ الأيام) المطلوبة فقط
    """
    employee_preferance = []
    base_cols = ['user_id', 'name', 'role']
    wanted_cols = base_cols + date_range

    for name in employee_names:
        record = df[df['name'] == name][wanted_cols].to_dict('records')
        employee_preferance.append(record)
    return employee_preferance


In [15]:
# def get_preferance_employees(df, employee_names: list):
#   employee_preferance= []

#   for name in employee_names:
#       employee_preferance.append(df[df['name'] == name].to_dict('records'))

#   return employee_preferance

In [36]:
get_preferance_employees(df_without_head, employee_names=['Emp E1D1#5', 'Emp E1D1#6', 'Emp E1D1#7'], date_range = ["2025-08-01", "2025-08-02", "2025-08-03", "2025-08-04", "2025-08-05", "2025-08-06", "2025-08-07"])

[[{'user_id': 6,
   'name': 'Emp E1D1#5',
   'role': 'Employee',
   '2025-08-01': 'night',
   '2025-08-02': 'night',
   '2025-08-03': 'afternoon',
   '2025-08-04': 'morning',
   '2025-08-05': 'morning',
   '2025-08-06': 'morning',
   '2025-08-07': 'morning'}],
 [{'user_id': 7,
   'name': 'Emp E1D1#6',
   'role': 'Employee',
   '2025-08-01': 'afternoon',
   '2025-08-02': 'morning',
   '2025-08-03': 'afternoon',
   '2025-08-04': 'morning',
   '2025-08-05': 'afternoon',
   '2025-08-06': 'night',
   '2025-08-07': 'night'}],
 [{'user_id': 8,
   'name': 'Emp E1D1#7',
   'role': 'Employee',
   '2025-08-01': 'morning',
   '2025-08-02': 'morning',
   '2025-08-03': 'night',
   '2025-08-04': 'morning',
   '2025-08-05': 'morning',
   '2025-08-06': 'afternoon',
   '2025-08-07': 'afternoon'}]]

### Create Parser by name you can add description of pydantic

In [17]:
def create_parser(employee_names: List[str], description: str, num_items: int,):

    pydeantic_string = f"""class Shift(BaseModel):
    shift_employee: List[Literal{employee_names}] = Field(..., description={repr(description)}, min_items={num_items}, max_items={num_items})\n"""

    print(pydeantic_string)

    compiled = compile(pydeantic_string, "<string>", "exec")
    exec_globals = {

    'BaseModel': BaseModel,
    'Field': Field,
    'List': List,
    'Literal': Literal,
    '__builtins__': __builtins__,
    }
    exec(compiled, exec_globals)

    EmployeesModel = exec_globals["Shift"]
    parser = PydanticOutputParser(pydantic_object=EmployeesModel)

    return parser

In [18]:
create_parser(employee_names = ['Emp E1D1#5', 'Emp E1D1#6', 'Emp E1D1#7'], description = 'choose employee for specific shift', num_items = 3)

class Shift(BaseModel):
    shift_employee: List[Literal['Emp E1D1#5', 'Emp E1D1#6', 'Emp E1D1#7']] = Field(..., description='choose employee for specific shift', min_items=3, max_items=3)



PydanticOutputParser(pydantic_object=<class 'Shift'>)

In [19]:
def get_employee_name_per_shift(df,day:str = data.columns[3]):
  employee_per_shift  = {}
  num_employee_per_shift = {}
  if day == df.columns[3]:
      for shift_time in ['morning', 'afternoon', 'night']:
          employee_shift = df[df[day] == shift_time]['name'].to_list()
          employee_per_shift[shift_time] = (employee_shift)
          num_employee_per_shift[shift_time] = len(employee_shift)
  else:

      for shift_time in ['morning', 'afternoon', 'night']:
          previous_day = df.columns[df.columns.get_loc(day) - 1]
          if shift_time == "morning":
              employee_shift = df[(df[day] == shift_time) & (df[previous_day] != 'night')]['name'].to_list()
              mask = (df[day] == shift_time) & (df[previous_day] == 'night')
              df.loc[mask, day] = "afternoon"


          if shift_time == "afternoon":
              employee_shift = df[df[day] == shift_time]['name'].to_list()
          else :
              employee_shift = df[df[day] == shift_time]['name'].to_list()

          employee_per_shift[shift_time] = (employee_shift)
          num_employee_per_shift[shift_time] = len(employee_shift)
  return employee_per_shift, num_employee_per_shift

In [20]:
get_employee_name_per_shift(df_without_head,day = data.columns[3])

({'morning': ['Emp E1D1#7',
   'Emp E1D1#11',
   'Emp E1D1#12',
   'Emp E1D1#14',
   'Emp E1D1#15',
   'Emp E1D1#17',
   'Emp E1D1#18',
   'Emp E1D1#19',
   'Emp E1D1#21',
   'Emp E1D1#22',
   'Emp E1D1#24',
   'Emp E1D1#28',
   'Emp E1D1#31',
   'Emp E1D1#34',
   'Emp E1D1#35',
   'Emp E1D1#38',
   'Emp E1D1#42',
   'Emp E1D1#43',
   'Emp E1D1#44',
   'Emp E1D1#48'],
  'afternoon': ['Emp E1D1#6',
   'Emp E1D1#9',
   'Emp E1D1#10',
   'Emp E1D1#16',
   'Emp E1D1#23',
   'Emp E1D1#25',
   'Emp E1D1#26',
   'Emp E1D1#29',
   'Emp E1D1#30',
   'Emp E1D1#36',
   'Emp E1D1#37',
   'Emp E1D1#40',
   'Emp E1D1#45',
   'Emp E1D1#46',
   'Emp E1D1#47',
   'Emp E1D1#49',
   'Emp E1D1#50'],
  'night': ['Emp E1D1#5',
   'Emp E1D1#8',
   'Emp E1D1#13',
   'Emp E1D1#20',
   'Emp E1D1#27',
   'Emp E1D1#32',
   'Emp E1D1#33',
   'Emp E1D1#39',
   'Emp E1D1#41']},
 {'morning': 20, 'afternoon': 17, 'night': 9})

In [21]:
# get_employee_name_per_shift(df_without_head,day = 'Monday')

In [22]:
# df_without_head['Monday'].value_counts()

### Call LLM

In [ ]:
def call_llm(parser, max_employee_shift: str , num_items: int, employee_preferance, day, shift):
  llm = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct', api_key='YOUR_GROQ_API_KEY')

  "  # llm = ChatGroq(model='llama3-70b-8192', api_key='YOUR_GROQ_API_KEY')
",

          "  ##llm= ChatOpenAI(model='gpt-4o',api_key= "YOUR_OPENAI_API_KEY")
",
  prompt = PromptTemplate(
      input_variables=['target_number', 'employee_list', 'preferance'],
      template=llm_prompt,
      partial_variables={'format_instructions': parser.get_format_instructions()}
  )
  chain = prompt | llm | parser


  result = chain.invoke({
    "employee_list": max_employee_shift,
    "target_number": num_items,
    'preferance': employee_preferance,
    'selected_day': day,
    'required_shift': shift
  })

  return result

In [24]:
def order_shift_list(shift_list, priority_names):
    """يرتب الموظفين بحيث اللي ليهم preference في الأول"""
    # الأولوية: اللي في priority_names (مع الحفاظ على ترتيبهم)، ثم الباقي
    shift_set = set(shift_list)
    priority_in_shift = [name for name in priority_names if name in shift_set]
    rest = [name for name in shift_list if name not in priority_in_shift]
    return priority_in_shift + rest


In [25]:
def get_employee_per_day(df , day:str):
    if day == df.columns[3]:
        previous_day = None
    else:
        previous_day = list(df.columns)[df.columns.get_loc(day) - 1]

    print(previous_day)

    employee_per_shift, num_employee_per_shift = get_employee_name_per_shift(df ,day = day)
    print(employee_per_shift)
    print(num_employee_per_shift)

    sorted_num_employee_per_shift = dict(sorted(num_employee_per_shift.items(),
                                                key=lambda item: item[1],
                                                reverse=True))
    map_idx_to_shift = {k:v for k, v in enumerate(sorted_num_employee_per_shift.keys())}

    print(sorted_num_employee_per_shift)
    print(map_idx_to_shift)


    max_employee_shift = employee_per_shift[map_idx_to_shift[0]]
    max_employee_shift = order_shift_list(max_employee_shift, dic_pref[day])

    range = get_week_dates_from_day(df, day)
    employee_preferance = get_preferance_employees(df, max_employee_shift, range)
    num_items = (df['name'].count()) // 3


    description = "choose employee for specific shift"

    parser = create_parser(max_employee_shift, description, num_items)
    result = call_llm(parser, max_employee_shift, num_items, employee_preferance,day,map_idx_to_shift[0])

    print(result)

    ## done first shift
    max_shift_list = result.model_dump()['shift_employee']



    print(f"{map_idx_to_shift[0]} of {day}", max_shift_list)
    print(num_items, len(max_shift_list))

    ############################################################################################################

    not_choose_shift = list(set(max_employee_shift) - set(max_shift_list))

    next_shift = map_idx_to_shift[1]
    print('======>>>>', next_shift)
    print(next_shift)

    if next_shift == 'morning' and previous_day is not None:

        mask = df['name'].isin(not_choose_shift) & (df[previous_day] != 'night')
        df.loc[mask, day] = next_shift

        print(df.loc[mask, day])
        mask = df['name'].isin(not_choose_shift) & (df[previous_day] == 'night')
        df.loc[mask, day] = map_idx_to_shift[2]

        next_employee_shift = df[df[day] == next_shift]['name'].to_list()
        if len(next_employee_shift) < num_items:
            n = num_items - len(next_employee_shift)
            mask3 = (df[previous_day] != "night") & (df[day] != "morning") & (df[day] != map_idx_to_shift[0])
            temp = df[mask3]
            selected = temp.sample(n, random_state=42)
            df.loc[selected.index, day] = next_shift


    else:
        print('------------')
        print(day, not_choose_shift)
        df.loc[df['name'].isin(not_choose_shift) , day] = next_shift


    print("+++++++++\n", df[day].value_counts(), "\n++++++++++++")
    next_employee_shift = df[df[day] == next_shift]['name'].to_list()
    next_employee_shift = order_shift_list(next_employee_shift, dic_pref[day])

    employee_preferance = get_preferance_employees(df,next_employee_shift, range)
    parser = create_parser(next_employee_shift, description, num_items)
    # print(parser)
    result = call_llm(parser, next_employee_shift, num_items, employee_preferance,day,map_idx_to_shift[1])
    print(result)
    max_shift_list = result.model_dump()['shift_employee']
    print(f"{map_idx_to_shift[1]} of {day}", max_shift_list)
    print(num_items, len(max_shift_list))

    not_choose_shift = set(next_employee_shift) - set(max_shift_list)




    ######################################################################################
    if map_idx_to_shift[2] == 'morning' and previous_day is not None:
        mask1 = df['name'].isin(not_choose_shift) & (df[previous_day] != 'night') #emp -> morning but not night
        df.loc[mask1, day] = map_idx_to_shift[2]
        mask2 = df['name'].isin(not_choose_shift) & (df[previous_day] == 'night')
        num_emp_mask = mask2.sum()



        mask3 = (df[previous_day] != "night") & (df[day] != "morning")

        temp = df[mask3]
        selected = temp.sample(num_emp_mask, random_state=42)

        val_temp = df.loc[selected.index, day].to_list()

        df.loc[selected.index, day] = map_idx_to_shift[2]


        df.loc[mask2, day] = val_temp

    else:
        df.loc[df['name'].isin(not_choose_shift) , day] = map_idx_to_shift[2]

    print("+++++++++\n", df[day].value_counts(), "\n++++++++++++")

    return df


In [26]:
day1 = get_employee_per_day(df = df_without_head, day = data.columns[3])

# day2 = get_employee_per_day(df = day1, day = data.columns[4])

# day3 = get_employee_per_day(df = day2, day = 'Tuesday')

# day4 = get_employee_per_day(df = day3, day = 'Wednesday')

# day5 = get_employee_per_day(df = day4, day = 'Thursday')

# day6 = get_employee_per_day(df = day5, day = 'Friday')

# day7 = get_employee_per_day(df = day6, day = 'Saturday')

None
{'morning': ['Emp E1D1#7', 'Emp E1D1#11', 'Emp E1D1#12', 'Emp E1D1#14', 'Emp E1D1#15', 'Emp E1D1#17', 'Emp E1D1#18', 'Emp E1D1#19', 'Emp E1D1#21', 'Emp E1D1#22', 'Emp E1D1#24', 'Emp E1D1#28', 'Emp E1D1#31', 'Emp E1D1#34', 'Emp E1D1#35', 'Emp E1D1#38', 'Emp E1D1#42', 'Emp E1D1#43', 'Emp E1D1#44', 'Emp E1D1#48'], 'afternoon': ['Emp E1D1#6', 'Emp E1D1#9', 'Emp E1D1#10', 'Emp E1D1#16', 'Emp E1D1#23', 'Emp E1D1#25', 'Emp E1D1#26', 'Emp E1D1#29', 'Emp E1D1#30', 'Emp E1D1#36', 'Emp E1D1#37', 'Emp E1D1#40', 'Emp E1D1#45', 'Emp E1D1#46', 'Emp E1D1#47', 'Emp E1D1#49', 'Emp E1D1#50'], 'night': ['Emp E1D1#5', 'Emp E1D1#8', 'Emp E1D1#13', 'Emp E1D1#20', 'Emp E1D1#27', 'Emp E1D1#32', 'Emp E1D1#33', 'Emp E1D1#39', 'Emp E1D1#41']}
{'morning': 20, 'afternoon': 17, 'night': 9}
{'morning': 20, 'afternoon': 17, 'night': 9}
{0: 'morning', 1: 'afternoon', 2: 'night'}
class Shift(BaseModel):
    shift_employee: List[Literal['Emp E1D1#38', 'Emp E1D1#7', 'Emp E1D1#11', 'Emp E1D1#12', 'Emp E1D1#14', 'Emp E

In [27]:
day2 = get_employee_per_day(df = day1, day = data.columns[4])

2025-08-01
{'morning': ['Emp E1D1#6', 'Emp E1D1#14', 'Emp E1D1#23', 'Emp E1D1#25', 'Emp E1D1#26', 'Emp E1D1#37', 'Emp E1D1#38', 'Emp E1D1#43'], 'afternoon': ['Emp E1D1#7', 'Emp E1D1#8', 'Emp E1D1#9', 'Emp E1D1#10', 'Emp E1D1#11', 'Emp E1D1#12', 'Emp E1D1#16', 'Emp E1D1#17', 'Emp E1D1#20', 'Emp E1D1#21', 'Emp E1D1#22', 'Emp E1D1#24', 'Emp E1D1#27', 'Emp E1D1#28', 'Emp E1D1#29', 'Emp E1D1#34', 'Emp E1D1#35', 'Emp E1D1#36', 'Emp E1D1#39', 'Emp E1D1#41', 'Emp E1D1#42', 'Emp E1D1#44', 'Emp E1D1#46', 'Emp E1D1#47', 'Emp E1D1#48', 'Emp E1D1#49', 'Emp E1D1#50'], 'night': ['Emp E1D1#5', 'Emp E1D1#13', 'Emp E1D1#15', 'Emp E1D1#18', 'Emp E1D1#19', 'Emp E1D1#30', 'Emp E1D1#31', 'Emp E1D1#32', 'Emp E1D1#33', 'Emp E1D1#40', 'Emp E1D1#45']}
{'morning': 8, 'afternoon': 27, 'night': 11}
{'afternoon': 27, 'night': 11, 'morning': 8}
{0: 'afternoon', 1: 'night', 2: 'morning'}
class Shift(BaseModel):
    shift_employee: List[Literal['Emp E1D1#7', 'Emp E1D1#9', 'Emp E1D1#11', 'Emp E1D1#12', 'Emp E1D1#21', '

In [28]:
# نفترض الأعمدة من 3 وما بعدهم هي أيام الشهر
day_columns = list(df_without_head.columns[3:])

df_current = df_without_head.copy()

for day_col in day_columns:
    max_attempts = 10
    attempt = 0
    while attempt < max_attempts:
        try:
            df_current = get_employee_per_day(df=df_current, day=day_col)
            break  # لو نفذ بنجاح اخرج من الloop بتاعت المحاولات
        except Exception as e:
            attempt += 1
            print(f"Error on {day_col} (attempt {attempt}): {e}")
            if attempt == max_attempts:
                print(f"Failed to process {day_col} after {max_attempts} attempts. Skipping...")
                # ممكن هنا تسجل اليوم ده في قائمة فشل مثلاً لو عايز تراجعه بعدين
    if max_attempts == attempt:
        break

df_current

None
{'morning': ['Emp E1D1#7', 'Emp E1D1#11', 'Emp E1D1#12', 'Emp E1D1#14', 'Emp E1D1#15', 'Emp E1D1#17', 'Emp E1D1#18', 'Emp E1D1#19', 'Emp E1D1#21', 'Emp E1D1#22', 'Emp E1D1#24', 'Emp E1D1#28', 'Emp E1D1#31', 'Emp E1D1#34', 'Emp E1D1#38'], 'afternoon': ['Emp E1D1#6', 'Emp E1D1#9', 'Emp E1D1#10', 'Emp E1D1#16', 'Emp E1D1#23', 'Emp E1D1#25', 'Emp E1D1#26', 'Emp E1D1#29', 'Emp E1D1#30', 'Emp E1D1#35', 'Emp E1D1#36', 'Emp E1D1#37', 'Emp E1D1#40', 'Emp E1D1#42', 'Emp E1D1#43'], 'night': ['Emp E1D1#5', 'Emp E1D1#8', 'Emp E1D1#13', 'Emp E1D1#20', 'Emp E1D1#27', 'Emp E1D1#32', 'Emp E1D1#33', 'Emp E1D1#39', 'Emp E1D1#41', 'Emp E1D1#44', 'Emp E1D1#45', 'Emp E1D1#46', 'Emp E1D1#47', 'Emp E1D1#48', 'Emp E1D1#49', 'Emp E1D1#50']}
{'morning': 15, 'afternoon': 15, 'night': 16}
{'night': 16, 'morning': 15, 'afternoon': 15}
{0: 'night', 1: 'morning', 2: 'afternoon'}
class Shift(BaseModel):
    shift_employee: List[Literal['Emp E1D1#5', 'Emp E1D1#13', 'Emp E1D1#8', 'Emp E1D1#20', 'Emp E1D1#27', 'Emp 

,user_id,name,role,2025-08-01,2025-08-02,2025-08-03,2025-08-04,2025-08-05,2025-08-06,2025-08-07,...,2025-08-22,2025-08-23,2025-08-24,2025-08-25,2025-08-26,2025-08-27,2025-08-28,2025-08-29,2025-08-30,2025-08-31
4,6,Emp E1D1#5,Employee,night,night,afternoon,morning,morning,morning,morning,...,morning,night,afternoon,morning,night,afternoon,morning,morning,morning,morning
5,7,Emp E1D1#6,Employee,afternoon,morning,morning,morning,afternoon,morning,morning,...,afternoon,afternoon,morning,afternoon,afternoon,afternoon,night,night,night,night
6,8,Emp E1D1#7,Employee,morning,morning,morning,morning,morning,morning,morning,...,morning,night,night,night,night,afternoon,morning,afternoon,morning,morning
7,9,Emp E1D1#8,Employee,night,afternoon,afternoon,afternoon,afternoon,morning,night,...,afternoon,night,afternoon,night,afternoon,afternoon,night,afternoon,afternoon,night
8,10,Emp E1D1#9,Employee,afternoon,morning,morning,morning,afternoon,afternoon,afternoon,...,afternoon,afternoon,morning,night,afternoon,morning,morning,morning,morning,afternoon
9,11,Emp E1D1#10,Employee,afternoon,afternoon,night,night,afternoon,afternoon,morning,...,afternoon,afternoon,morning,afternoon,night,afternoon,night,afternoon,afternoon,afternoon
10,12,Emp E1D1#11,Employee,morning,afternoon,morning,night,afternoon,afternoon,afternoon,...,afternoon,morning,afternoon,afternoon,night,afternoon,night,afternoon,afternoon,morning
11,13,Emp E1D1#12,Employee,morning,afternoon,night,night,afternoon,morning,morning,...,night,afternoon,night,afternoon,night,night,afternoon,night,afternoon,morning
12,14,Emp E1D1#13,Employee,night,night,afternoon,morning,afternoon,morning,morning,...,afternoon,afternoon,night,afternoon,afternoon,afternoon,afternoon,afternoon,morning,night
13,15,Emp E1D1#14,Employee,morning,morning,morning,afternoon,morning,morning,night,...,night,afternoon,morning,morning,afternoon,night,night,afternoon,afternoon,night


In [39]:
with open("/home/elbahnasy/CodingWorkspace/Hands-On-LLMS/notebooks/employees_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

issuer_id = data.get("issuer_id")

issuer_id

2

In [40]:
def df_to_json_format(df, year, month):
    employees_json = []
    date_columns = [col for col in df.columns if col.count('-') == 2]
    for _, row in df.iterrows():
        preferences = []
        for date in date_columns:
            shift = row[date]
            if shift and shift not in ["", "off"]:
                preferences.append({"date": date, "shiftType": shift})
        emp_json = {
            "user_id": int(row["user_id"]),
            "name": row["name"],
            "role": row["role"],
            "preferences": preferences,
            "status": None  # or row.get("status", None) if available
        }
        employees_json.append(emp_json)
    output = {
        "issuer_id":data.get("issuer_id"),
        "employees_count":data.get("employees_count"),
        "department_id": data.get("department_id"),
        "enterprise_id": data.get("enterprise_id"),
        "year": year,
        "month": month,
        "employees": employees_json,
    }
    return output

# Example usage:
json_output = df_to_json_format(df_current, year, month)
# To save to file:
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(json_output, f, ensure_ascii=False, indent=2)

In [29]:

for i in df_current.columns[3:]:
    print(df_current[i].value_counts(), "\n\n")

2025-08-01
afternoon    16
night        15
morning      15
Name: count, dtype: int64 


2025-08-02
night        16
morning      15
afternoon    15
Name: count, dtype: int64 


2025-08-03
morning      16
afternoon    15
night        15
Name: count, dtype: int64 


2025-08-04
afternoon    16
morning      15
night        15
Name: count, dtype: int64 


2025-08-05
night        16
morning      15
afternoon    15
Name: count, dtype: int64 


2025-08-06
morning      16
afternoon    15
night        15
Name: count, dtype: int64 


2025-08-07
morning      16
night        15
afternoon    15
Name: count, dtype: int64 


2025-08-08
afternoon    16
night        15
morning      15
Name: count, dtype: int64 


2025-08-09
morning      16
night        15
afternoon    15
Name: count, dtype: int64 


2025-08-10
morning      16
afternoon    15
night        15
Name: count, dtype: int64 


2025-08-11
morning      16
night        15
afternoon    15
Name: count, dtype: int64 


2025-08-12
night        16
morni

In [32]:
# Check for each date column if 'morning' follows 'night' in the previous day
violations = []
date_columns = list(df_current.columns[3:])  # Assuming first 3 columns are not dates

for idx in range(1, len(date_columns)):
    current_col = date_columns[idx]
    prev_col = date_columns[idx - 1]
    mask = (df_current[current_col] == "morning") & (df_current[prev_col] == "night")
    if mask.any():
        violations.append({
            "current_day": current_col,
            "previous_day": prev_col,
            "employees": df_current.loc[mask, "name"].tolist()
        })

violations  

[]

In [33]:


# Check for each date column if 'morning' follows 'night' in the previous day
violations = []
date_columns = list(df_without_head.columns[3:])  # Assuming first 3 columns are not dates

for idx in range(1, len(date_columns)):
    current_col = date_columns[idx]
    prev_col = date_columns[idx - 1]
    mask = (df_without_head[current_col] == "morning") & (df_without_head[prev_col] == "night")
    if mask.any():
        violations.append({
            "current_day": current_col,
            "previous_day": prev_col,
            "employees": df_without_head.loc[mask, "name"].tolist()
        })

violations  

[{'current_day': '2025-08-03',
  'previous_day': '2025-08-02',
  'employees': ['Emp E1D1#16', 'Emp E1D1#31', 'Emp E1D1#32', 'Emp E1D1#46']},
 {'current_day': '2025-08-04',
  'previous_day': '2025-08-03',
  'employees': ['Emp E1D1#7', 'Emp E1D1#25', 'Emp E1D1#28']},
 {'current_day': '2025-08-05',
  'previous_day': '2025-08-04',
  'employees': ['Emp E1D1#10',
   'Emp E1D1#11',
   'Emp E1D1#18',
   'Emp E1D1#19',
   'Emp E1D1#21',
   'Emp E1D1#22',
   'Emp E1D1#30',
   'Emp E1D1#32',
   'Emp E1D1#33',
   'Emp E1D1#35',
   'Emp E1D1#42']},
 {'current_day': '2025-08-06',
  'previous_day': '2025-08-05',
  'employees': ['Emp E1D1#16', 'Emp E1D1#29', 'Emp E1D1#34']},
 {'current_day': '2025-08-07',
  'previous_day': '2025-08-06',
  'employees': ['Emp E1D1#17', 'Emp E1D1#18', 'Emp E1D1#24', 'Emp E1D1#37']},
 {'current_day': '2025-08-08',
  'previous_day': '2025-08-07',
  'employees': ['Emp E1D1#6',
   'Emp E1D1#40',
   'Emp E1D1#41',
   'Emp E1D1#42',
   'Emp E1D1#48',
   'Emp E1D1#50']},
 {'cur